In [ ]:
!git clone https://github.com/Mr-Patty/bimodal-emotion-recognition
!pip install soundfile

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [ ]:
!cp drive/'My Drive'/EmotionRecognition/bimodal-emotion-recognition.tar.gz bimodal-emotion-recognition/
%cd bimodal-emotion-recognition
!cp drive/'My Drive'/EmotionRecognition/df_prep.csv .
!tar -C . -xzf bimodal-emotion-recognition.tar.gz
!rm bimodal-emotion-recognition.tar.gz

In [1]:
import os
import glob
import torch
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader
from utils.hparams import HParam
from datasets.dataloader import create_dataloader, create_dataset

from utils.utils import read_wav_np
import wave
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import math
import librosa
import json
import wave
import sys
import pickle
import sklearn

import urllib.request

import librosa.display
import scipy, matplotlib.pyplot as plt, IPython.display as ipd

from scipy.io import wavfile

import librosa.display
import soundfile as sf

import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import os
import matplotlib.style as ms
from tqdm import tqdm
import random
import logging
import time

from utils.train import *
from models import *
ms.use('seaborn-muted')
%matplotlib inline

In [2]:
hp = HParam('config/default.yaml')
with open('config/default.yaml', 'r') as f:
    hp_str = ''.join(f.readlines())

In [3]:
pt_dir = os.path.join(hp.log.chkpt_dir, 'AudioModel')
log_dir = os.path.join(hp.log.log_dir, 'AudioModel')
os.makedirs(pt_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

In [4]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(log_dir,
            '%s-%d.log' % ('AudioModel', time.time()))),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

In [5]:
assert hp.audio.hop_length == 256, \
    'hp.audio.hop_length must be equal to 256, got %d' % hp.audio.hop_length
assert hp.data.train != '' \
    'hp.data.train and hp.data.validation can\'t be empty: please fix %s' % 'config/default.yaml'

In [6]:
trainset = create_dataset(hp, 'df_prep.csv', True)
# train_loader = DataLoader(dataset, num_workers=1, shuffle=False,
#                               batch_size=1)

In [7]:
# for i in tqdm(trainloader):
#     print(2)

In [10]:
Batch = 1
model = AttentionRNNModel(batch_size=Batch, output_size=7, hidden_size=32, embedding_length=80)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
train_loss, val_loss, best_model = train(model, trainset, device, batch_size=Batch, n_epoch=1, validation_split=.95)